# NORTHWIND DATABASE

El **objetivo** de este ejercicio es generar una serie de consultas SQL que permitan extraer información relevante de la base de datos y luego exportar los resultados para construir dashboards o visualizaciones que proporcionen insights valiosos para la toma de decisiones empresariales.

La base de datos Northwind es una base de datos de ejemplo clásica que simula las operaciones de una empresa minorista. Contiene tablas relacionales que representan diferentes aspectos del negocio, como productos, clientes, pedidos y empleados. Utilizando consultas SQL, podrás explorar y analizar estos datos para comprender mejor el desempeño del negocio en diferentes áreas.

Una vez que hayas realizado las consultas SQL y obtenido los resultados, podrás utilizarlos para herramientas de visualización de datos como Tableau, Power BI, Streamlit o cualquier otra herramienta de tu elección. 

![img](./img/northwind-er-diagram.png)

Tendrás que elegir uno de los retos a resolver:

### **1. Análisis de Ventas por Región**
Analiza las ventas de productos por región. El objetivo es identificar las regiones que generan más ingresos y las tendencias de ventas a lo largo del tiempo en esas regiones. 
Pista: Utiliza gráficos de barras y líneas para mostrar la evolución de las ventas, así como mapas geográficos para visualizar la distribución de ingresos por región.

In [28]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///data/northwind.db', echo=False)
connection = engine.connect()

In [29]:
import pandas as pd

consulta = """
SELECT datetime(o.OrderDate) as OrderDate, o.ShipRegion, od.Quantity*od.UnitPrice*(1-od.Discount) AS Precio_total, p.ProductName
FROM Orders o
INNER JOIN [Order Details] od ON od.OrderID == o.OrderID
INNER JOIN Products p ON p.ProductID = od.ProductID
"""

data_df = pd.read_sql(consulta, con=connection)
data_df

,OrderDate,ShipRegion,Precio_total,ProductName
0,2016-07-04 00:00:00,Western Europe,168.00,Queso Cabrales
1,2016-07-04 00:00:00,Western Europe,98.00,Singaporean Hokkien Fried Mee
2,2016-07-04 00:00:00,Western Europe,174.00,Mozzarella di Giovanni
3,2016-07-05 00:00:00,Western Europe,167.40,Tofu
4,2016-07-05 00:00:00,Western Europe,1696.00,Manjimup Dried Apples
...,...,...,...,...
609278,2023-01-10 05:40:51,British Isles,806.00,Ikura
609279,2023-01-10 05:40:51,British Isles,216.00,Spegesild
609280,2023-01-10 05:40:51,British Isles,93.69,Gumbär Gummibärchen
609281,2023-01-10 05:40:51,British Isles,1053.60,Schoggi Schokolade


In [30]:
data_df.dtypes

OrderDate        object
ShipRegion       object
Precio_total    float64
ProductName      object
dtype: object

In [31]:
data_df["OrderDate"] = pd.to_datetime(data_df['OrderDate'])
data_df.set_index("OrderDate", inplace=True)

In [32]:
data_df.index

DatetimeIndex(['2016-07-04 00:00:00', '2016-07-04 00:00:00',
               '2016-07-04 00:00:00', '2016-07-05 00:00:00',
               '2016-07-05 00:00:00', '2016-07-08 00:00:00',
               '2016-07-08 00:00:00', '2016-07-08 00:00:00',
               '2016-07-08 00:00:00', '2016-07-08 00:00:00',
               ...
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51'],
              dtype='datetime64[ns]', name='OrderDate', length=609283, freq=None)

In [33]:
data_df.head()

,ShipRegion,Precio_total,ProductName
OrderDate,,,
2016-07-04,Western Europe,168.0,Queso Cabrales
2016-07-04,Western Europe,98.0,Singaporean Hokkien Fried Mee
2016-07-04,Western Europe,174.0,Mozzarella di Giovanni
2016-07-05,Western Europe,167.4,Tofu
2016-07-05,Western Europe,1696.0,Manjimup Dried Apples


In [34]:
data_df["ShipRegion"].unique()

array(['Western Europe', 'South America', 'Central America',
       'North America', 'Northern Europe', 'Scandinavia',
       'Southern Europe', 'British Isles', 'Eastern Europe'], dtype=object)

In [35]:
reg_prod_total = data_df.groupby(["ShipRegion"]).resample("BYE").sum("Precio_total")
reg_prod_total

Precio_total
ShipRegion     OrderDate               
British Isles  2012-12-31    1697832.07
               2013-12-31    3128226.90
               2014-12-31    3884546.08
               2015-12-31    3511450.44
               2016-12-30    3574917.46
...                                 ...
Western Europe 2019-12-31   11526084.85
               2020-12-31   12029185.51
               2021-12-31   11217097.30
               2022-12-30   11066267.48
               2023-12-29    9940536.04

[108 rows x 1 columns]

In [36]:
reg_prod_total.reset_index(inplace=True)
reg_prod_total

,ShipRegion,OrderDate,Precio_total
0,British Isles,2012-12-31,1697832.07
1,British Isles,2013-12-31,3128226.90
2,British Isles,2014-12-31,3884546.08
3,British Isles,2015-12-31,3511450.44
4,British Isles,2016-12-30,3574917.46
...,...,...,...
103,Western Europe,2019-12-31,11526084.85
104,Western Europe,2020-12-31,12029185.51
105,Western Europe,2021-12-31,11217097.30
106,Western Europe,2022-12-30,11066267.48


In [37]:
reg_prod_total["Year"] = reg_prod_total["OrderDate"].dt.year
reg_prod_total

,ShipRegion,OrderDate,Precio_total,Year
0,British Isles,2012-12-31,1697832.07,2012
1,British Isles,2013-12-31,3128226.90,2013
2,British Isles,2014-12-31,3884546.08,2014
3,British Isles,2015-12-31,3511450.44,2015
4,British Isles,2016-12-30,3574917.46,2016
...,...,...,...,...
103,Western Europe,2019-12-31,11526084.85,2019
104,Western Europe,2020-12-31,12029185.51,2020
105,Western Europe,2021-12-31,11217097.30,2021
106,Western Europe,2022-12-30,11066267.48,2022


In [38]:
import plotly.express as px

fig = px.line(reg_prod_total, x="Year", y='Precio_total', color="ShipRegion")
fig.show()

In [63]:
consulta = """
SELECT strftime('%Y', OrderDate) AS Year, 
       strftime('%m', OrderDate) AS Month,
       ShipRegion AS Region, 
       ProductName AS Product, 
       SUM(Precio_total) As Total
FROM (
       SELECT datetime(o.OrderDate) as OrderDate, o.ShipRegion, od.Quantity*od.UnitPrice*(1-od.Discount) AS Precio_total, p.ProductName
       FROM Orders o
       INNER JOIN [Order Details] od ON od.OrderID == o.OrderID
       INNER JOIN Products p ON p.ProductID = od.ProductID
)
GROUP BY Year, Month, Region, Product
ORDER BY 5 DESC
"""

data_df = pd.read_sql(consulta, con=connection)
data_df

,Year,Month,Region,Product,Total
0,2014,12,Western Europe,Côte de Blaye,222130.5
1,2015,08,Western Europe,Côte de Blaye,220022.5
2,2016,12,Western Europe,Côte de Blaye,209746.0
3,2013,07,Western Europe,Côte de Blaye,209482.5
4,2018,08,Western Europe,Côte de Blaye,192882.0
...,...,...,...,...,...
86605,2018,09,Scandinavia,Guaraná Fantástica,4.5
86606,2018,10,Northern Europe,Guaraná Fantástica,4.5
86607,2021,05,Eastern Europe,Guaraná Fantástica,4.5
86608,2023,08,Scandinavia,Guaraná Fantástica,4.5


In [117]:
product_profit = data_df.groupby(["Product","Region"]).agg(meanPrice=("Total", "mean"), stdPrice=("Total","std")).sort_values("meanPrice", ascending=False)
product_profit

meanPrice      stdPrice
Product                 Region                                      
Côte de Blaye           Western Europe   120245.512500  35830.588553
                        North America     71226.975625  24537.409206
                        South America     70392.475000  26489.913360
Thüringer Rostbratwurst Western Europe    55366.795390  15763.042641
Mishi Kobe Niku         Western Europe    44114.744118  12931.501600
...                                                ...           ...
Geitost                 Central America     223.130769    119.268353
                        Northern Europe     180.480315    108.326253
                        Scandinavia         153.272727     97.388931
Guaraná Fantástica      Eastern Europe      148.616883    108.727747
Geitost                 Eastern Europe       94.202899     55.984656

[693 rows x 2 columns]

In [70]:
region = "Western Europe" # North America

region_profit = data_df[data_df["Region"] == region].groupby(["Product"]).agg(meanPrice=("Total", "mean"), stdPrice=("Total","std")).sort_values("meanPrice", ascending=False)
region_profit

,meanPrice,stdPrice
Product,,
Côte de Blaye,71226.975625,24537.409206
Thüringer Rostbratwurst,30920.014743,11221.340795
Mishi Kobe Niku,24555.264706,7952.357134
Sir Rodney's Marmalade,21135.496765,7044.242681
Carnarvon Tigers,16778.216912,6228.778044
...,...,...
Tourtière,1932.552132,678.237755
Filo Mix,1760.808824,590.562852
Konbu,1628.426471,571.066622


In [71]:
fig = px.bar(region_profit, y='meanPrice', error_y="stdPrice")
fig.show()

### **2. Análisis de Desempeño de Productos**
Analiza el desempeño de los productos de la empresa. Examina las ventas de productos por categoría, identificando los productos más vendidos y los menos vendidos. 

Pista: Utiliza gráficos de barras y de tarta para visualizar la participación de mercado de cada categoría y los ingresos generados por los productos individuales.

In [ ]:
# !pip install 'ibis-framework[sqlite]'

In [15]:
import ibis

# Para ejecución interactiva
# ibis.options.interactive = True

con = ibis.sqlite.connect('data/northwind.db')

In [84]:
categories = con.table("Categories")
products = con.table("Products")

joined_products = categories.join(products, "CategoryID")
joined_products

r0 := DatabaseTable: Categories
  CategoryID   int64
  CategoryName string
  Description  string
  Picture      binary

r1 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

JoinChain[r0]
  JoinLink[inner, r1]
    r0.CategoryID == r1.CategoryID
  values:
    CategoryID:      r0.CategoryID
    CategoryName:    r0.CategoryName
    Description:     r0.Description
    Picture:         r0.Picture
    ProductID:       r1.ProductID
    ProductName:     r1.ProductName
    SupplierID:      r1.SupplierID
    QuantityPerUnit: r1.QuantityPerUnit
    UnitPrice:       r1.UnitPrice
    UnitsInStock:    r1.UnitsInStock
    UnitsOnOrder:    r1.UnitsOnOrder
    ReorderLevel:    r1.ReorderLevel
    Discontinued:    r1.Discontinued

In [103]:
category_df = joined_products.select(["CategoryName", "ProductName"]).to_pandas()
category_df.rename(columns={"ProductName": "Product"}, inplace=True)
category_df.set_index("Product", inplace=True)

In [118]:
product_profit.reset_index(inplace=True)
product_profit.set_index("Product", inplace=True)

In [119]:
product_profit

,Region,meanPrice,stdPrice
Product,,,
Côte de Blaye,Western Europe,120245.512500,35830.588553
Côte de Blaye,North America,71226.975625,24537.409206
Côte de Blaye,South America,70392.475000,26489.913360
Thüringer Rostbratwurst,Western Europe,55366.795390,15763.042641
Mishi Kobe Niku,Western Europe,44114.744118,12931.501600
...,...,...,...
Geitost,Central America,223.130769,119.268353
Geitost,Northern Europe,180.480315,108.326253
Geitost,Scandinavia,153.272727,97.388931


In [122]:
category_df.join(product_profit).sort_values("meanPrice", ascending=False).head(10)

,CategoryName,Region,meanPrice,stdPrice
Product,,,,
Côte de Blaye,Beverages,Western Europe,120245.512500,35830.588553
Côte de Blaye,Beverages,North America,71226.975625,24537.409206
Côte de Blaye,Beverages,South America,70392.475000,26489.913360
Thüringer Rostbratwurst,Meat/Poultry,Western Europe,55366.795390,15763.042641
Mishi Kobe Niku,Meat/Poultry,Western Europe,44114.744118,12931.501600
Côte de Blaye,Beverages,Southern Europe,37932.201493,16579.994957
Sir Rodney's Marmalade,Confections,Western Europe,37618.663235,10658.487694
Côte de Blaye,Beverages,British Isles,36764.149254,18923.969776
Thüringer Rostbratwurst,Meat/Poultry,South America,33102.532382,12150.048244


In [169]:
type(category_df)

pandas.core.frame.DataFrame

In [171]:
type(joined_products)

ibis.expr.types.joins.Join

In [172]:
ibis.to_sql(joined_products)

```sql
SELECT
  "t2"."CategoryID",
  "t2"."CategoryName",
  "t2"."Description",
  "t2"."Picture",
  "t3"."ProductID",
  "t3"."ProductName",
  "t3"."SupplierID",
  "t3"."QuantityPerUnit",
  "t3"."UnitPrice",
  "t3"."UnitsInStock",
  "t3"."UnitsOnOrder",
  "t3"."ReorderLevel",
  "t3"."Discontinued"
FROM "Categories" AS "t2"
INNER JOIN "Products" AS "t3"
  ON "t2"."CategoryID" = "t3"."CategoryID"
```

In [131]:
con.table("Orders")

KeyError: <Type.DATETIME: 'DATETIME'>

In [133]:
orders = con.sql("SELECT CAST(OrderDate as Text), OrderID FROM Orders")
orders

SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text), OrderID FROM Orders
  schema:
    CAST(OrderDate as Text) string
    OrderID                 int64

In [134]:
orders.head(4).to_pandas()

,CAST(OrderDate as Text),OrderID
0,2016-07-04,10248
1,2016-07-05,10249
2,2016-07-08,10250
3,2016-07-08,10251


In [135]:
order_details = con.table("Order Details")

In [168]:
p_o_details = orders.join(order_details, ["OrderID"], how="inner").join(products, ["ProductID"]).join(categories, ["CategoryID"])
ibis.to_sql(p_o_details)

```sql
SELECT
  "t4"."CAST(OrderDate as Text)",
  "t4"."OrderID",
  "t5"."ProductID",
  "t5"."UnitPrice",
  "t5"."Quantity",
  "t5"."Discount",
  "t6"."ProductName",
  "t6"."SupplierID",
  "t6"."CategoryID",
  "t6"."QuantityPerUnit",
  "t6"."UnitPrice" AS "UnitPrice_right",
  "t6"."UnitsInStock",
  "t6"."UnitsOnOrder",
  "t6"."ReorderLevel",
  "t6"."Discontinued",
  "t7"."CategoryName",
  "t7"."Description",
  "t7"."Picture"
FROM (
  SELECT
    CAST(OrderDate AS TEXT),
    OrderID
  FROM Orders
) AS "t4"
INNER JOIN "Order Details" AS "t5"
  ON "t4"."OrderID" = "t5"."OrderID"
INNER JOIN "Products" AS "t6"
  ON "t5"."ProductID" = "t6"."ProductID"
INNER JOIN "Categories" AS "t7"
  ON "t6"."CategoryID" = "t7"."CategoryID"
```

In [150]:
p_o_details_agg = p_o_details.group_by(["CategoryName","ProductName"]).aggregate(
    num_orders=p_o_details["OrderID"].count(), 
    price=(p_o_details["Quantity"]*p_o_details["UnitPrice"]*(1-p_o_details["Discount"])).sum())
p_o_details_agg = p_o_details_agg.order_by(p_o_details_agg["num_orders"].desc(), p_o_details_agg["price"].desc())
p_o_details_agg

r0 := SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text), OrderID FROM Orders
  schema:
    CAST(OrderDate as Text) string
    OrderID                 int64

r1 := DatabaseTable: Order Details
  OrderID   !int64
  ProductID !int64
  UnitPrice !decimal
  Quantity  !int64
  Discount  !float64

r2 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

r3 := DatabaseTable: Categories
  CategoryID   int64
  CategoryName string
  Description  string
  Picture      binary

r4 := JoinChain[r0]
  JoinLink[inner, r1]
    r0.OrderID == r1.OrderID
  JoinLink[inner, r2]
    r1.ProductID == r2.ProductID
  JoinLink[inner, r3]
    r2.CategoryID == r3.CategoryID
  values:
    CAST(OrderDate as Text): r0['CAST(OrderDate as Text)']
    OrderID:                 r0.OrderID
    ProductID:               r1.ProductID
    UnitPrice:               r1.UnitPrice
    Quantity:                r1.Quantity
    Discount:                r1.Discount
    ProductName:             r2.ProductName
    SupplierID:              r2.SupplierID
    CategoryID:              r2.CategoryID
    QuantityPerUnit:         r2.QuantityPerUnit
    UnitPrice_right:         r2.UnitPrice
    UnitsInStock:            r2.UnitsInStock
    UnitsOnOrder:            r2.UnitsOnOrder
    ReorderLevel:            r2.ReorderLevel
    Discontinued:            r2.Discontinued
    CategoryName:            r3.CategoryName
    Description:             r3.Description
    Picture:                 r3.Picture

r5 := Aggregate[r4]
  groups:
    CategoryName: r4.CategoryName
    ProductName:  r4.ProductName
  metrics:
    num_orders: Count(r4.OrderID)
    price:      Sum(r4.Quantity * r4.UnitPrice * 1 - r4.Discount)

Sort[r5]
  desc r5.num_orders
  desc r5.price

In [151]:
p_o_details_agg.to_pandas()

,CategoryName,ProductName,num_orders,price
0,Condiments,Louisiana Hot Spiced Okra,8040,3504941
1,Confections,Teatime Chocolate Biscuits,8024,1889691.4200000037
2,Beverages,Outback Lager,8020,3064462.65
3,Confections,Sir Rodney's Marmalade,7999,16653807.36
4,Confections,Gumbär Gummibärchen,7999,6391018.984499998
...,...,...,...,...
72,Produce,Tofu,7834,4644738.99
73,Confections,Scottish Longbreads,7818,2528089
74,Meat/Poultry,Perth Pasties,7817,6561681.369999989
75,Dairy Products,Mascarpone Fabioli,7817,6381012.16


In [164]:
consulta= p_o_details_agg.group_by("CategoryName").aggregate(total_price=p_o_details_agg["price"].sum()).order_by(ibis.desc("total_price"))
consulta.to_pandas()

,CategoryName,total_price
0,Beverages,92163184.17999999
1,Confections,66337803.06499998
2,Meat/Poultry,64881147.969499916
3,Dairy Products,58018116.78500001
4,Condiments,55795126.78499997
5,Seafood,49921604.16749999
6,Produce,32701119.88000001
7,Grains/Cereals,28568530.3375


In [167]:
ibis.to_sql(consulta)

```sql
SELECT
  *
FROM (
  SELECT
    "t10"."CategoryName",
    SUM("t10"."price") AS "total_price"
  FROM (
    SELECT
      *
    FROM (
      SELECT
        "t8"."CategoryName",
        "t8"."ProductName",
        COUNT("t8"."OrderID") AS "num_orders",
        SUM((
          "t8"."Quantity" * "t8"."UnitPrice"
        ) * (
          1 - "t8"."Discount"
        )) AS "price"
      FROM (
        SELECT
          "t4"."CAST(OrderDate as Text)",
          "t4"."OrderID",
          "t5"."ProductID",
          "t5"."UnitPrice",
          "t5"."Quantity",
          "t5"."Discount",
          "t6"."ProductName",
          "t6"."SupplierID",
          "t6"."CategoryID",
          "t6"."QuantityPerUnit",
          "t6"."UnitPrice" AS "UnitPrice_right",
          "t6"."UnitsInStock",
          "t6"."UnitsOnOrder",
          "t6"."ReorderLevel",
          "t6"."Discontinued",
          "t7"."CategoryName",
          "t7"."Description",
          "t7"."Picture"
        FROM (
          SELECT
            CAST(OrderDate AS TEXT),
            OrderID
          FROM Orders
        ) AS "t4"
        INNER JOIN "Order Details" AS "t5"
          ON "t4"."OrderID" = "t5"."OrderID"
        INNER JOIN "Products" AS "t6"
          ON "t5"."ProductID" = "t6"."ProductID"
        INNER JOIN "Categories" AS "t7"
          ON "t6"."CategoryID" = "t7"."CategoryID"
      ) AS "t8"
      GROUP BY
        1,
        2
    ) AS "t9"
    ORDER BY
      "t9"."num_orders" DESC,
      "t9"."price" DESC
  ) AS "t10"
  GROUP BY
    1
) AS "t11"
ORDER BY
  "t11"."total_price" DESC
```

In [182]:
precio_por_categoría = consulta.to_pandas()

fig = px.bar(precio_por_categoría, x='CategoryName', y="total_price")
fig.show()

In [184]:
precio_producto= p_o_details_agg.to_pandas()

fig = px.bar(precio_producto, x='ProductName', y="price", color="CategoryName")
fig.show()

### **3. Análisis de Rentabilidad de Clientes**
Analiza la rentabilidad de los clientes de la empresa. Examina los ingresos generados por cada cliente, así como los costos asociados con el servicio o producto proporcionado. 

Pista: Utiliza gráficos de dispersión y tablas dinámicas para identificar los clientes más rentables y aquellos que requieren mayor atención.

### **4. Análisis de Inventarios y Proveedores**
Analiza los niveles de inventario y la gestión de proveedores de la empresa. Examina el inventario disponible por categoría de productos, así como el rendimiento de los proveedores en términos de puntualidad de entrega y calidad de los productos. 

Pista: Utiliza gráficos de líneas y tablas dinámicas para mostrar tendencias y comparar el desempeño de los proveedores.


### **5. Análisis de Comportamiento de Empleados**
Analiza el comportamiento de los empleados de la empresa. Examina las ventas realizadas por cada empleado, así como su eficiencia en el manejo de pedidos y su puntualidad en la entrega de productos. 

Pista: Utiliza gráficos de barra y de dispersión para comparar el desempeño de los empleados y identificar áreas de mejora.